In [1]:
import scanpy as sc
import anndata as an
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from typing import Optional, Union
import seaborn as sns
import scipy
import bbknn
from scipy.io import mmread

/slurm/home/yrd/liaolab/caohaoxue/anaconda3/envs/my_env/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import scib
from scib.metrics import silhouette_batch
from scib.metrics import kBET

In [3]:
from scipy import sparse

# E12

In [5]:
cyto = sc.read_h5ad('/slurm/home/yrd/liaolab/caohaoxue/embryo_work/benchmark/simulated_matrix/h5ad_format/E12f_CytO.h5ad')
spla = sc.read_h5ad('/slurm/home/yrd/liaolab/caohaoxue/embryo_work/benchmark/simulated_matrix/h5ad_format/E12f_splatter.h5ad')
simp = sc.read_h5ad('/slurm/home/yrd/liaolab/caohaoxue/embryo_work/benchmark/simulated_matrix/h5ad_format/E12f_splatter_simple.h5ad')
scde = sc.read_h5ad('/slurm/home/yrd/liaolab/caohaoxue/embryo_work/benchmark/simulated_matrix/h5ad_format/E12f_scDesign3.h5ad')
scva = sc.read_h5ad('/slurm/home/yrd/liaolab/caohaoxue/embryo_work/benchmark/simulated_matrix/h5ad_format/E12f_scVAEDer.h5ad')
real = sc.read_h5ad('/slurm/home/yrd/liaolab/caohaoxue/embryo_work/data_standard/Zygote_to_CS7.h5ad')

/slurm/home/yrd/liaolab/caohaoxue/anaconda3/envs/my_env/lib/python3.13/site-packages/anndata/_core/anndata.py:1756: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [6]:
real = real[real.obs.time == 'E12']
real.var = real.var.rename(columns = {'x':'features'})
real.var.index = real.var.features

In [7]:
adata_int = sc.concat([cyto,spla,simp,scde,scva],
                      join = 'outer',
                      fill_value = 0)

adata = sc.concat([real,adata_int],
                 label = 'source',
                 keys = ['real','simulated'],
                 index_unique = '-')

/slurm/home/yrd/liaolab/caohaoxue/anaconda3/envs/my_env/lib/python3.13/site-packages/anndata/_core/anndata.py:1756: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/slurm/home/yrd/liaolab/caohaoxue/anaconda3/envs/my_env/lib/python3.13/site-packages/anndata/_core/anndata.py:1756: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [8]:
adata_raw = sc.concat([real,adata_int],
                 label = 'source',
                 keys = ['real','simulated'],
                 index_unique = '-')

/slurm/home/yrd/liaolab/caohaoxue/anaconda3/envs/my_env/lib/python3.13/site-packages/anndata/_core/anndata.py:1756: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [9]:
adata.obs['stage'] = 'E12'
adata_raw.obs['stage'] = 'E12'

In [10]:
sc.tl.pca(adata_raw, svd_solver='arpack')
sc.pp.neighbors(adata_raw)
sc.tl.umap(adata_raw)

/slurm/home/yrd/liaolab/caohaoxue/anaconda3/envs/my_env/lib/python3.13/site-packages/sklearn/manifold/_spectral_embedding.py:329: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(


In [11]:
sc.tl.pca(adata, svd_solver='arpack')
sc.external.pp.bbknn(adata, batch_key='source')
sc.tl.umap(adata)

In [50]:
kbet_score_cyto = pcr_comparison(
    adata_raw[adata_raw.obs.time.isin(['E12','E12-CytOrigin'])],
    adata[adata.obs.time.isin(['E12','E12-CytOrigin'])],
    embed="X_umap",
    covariate='time'
)

kbet_score_scde = pcr_comparison(
    adata_raw[adata_raw.obs.time.isin(['E12','E12-scDesign3'])],
    adata[adata.obs.time.isin(['E12','E12-scDesign3'])],
    embed="X_umap",
    covariate='time'
)

kbet_score_scva = pcr_comparison(
    adata_raw[adata_raw.obs.time.isin(['E12','E12-scVAEDer'])],
    adata[adata.obs.time.isin(['E12','E12-scVAEDer'])],
    embed="X_umap",
    covariate='time'
)

kbet_score_spla = pcr_comparison(
    adata_raw[adata_raw.obs.time.isin(['E12','E12-Splatter'])],
    adata[adata.obs.time.isin(['E12','E12-Splatter'])],
    embed="X_umap",
    covariate='time'
)

kbet_score_simp = pcr_comparison(
    adata_raw[adata_raw.obs.time.isin(['E12','E12-Splatter-Simple'])],
    adata[adata.obs.time.isin(['E12','E12-Splatter-Simple'])],
    embed="X_umap",
    covariate='time'
)

Variance contribution increased after integration!
Setting PCR comparison score to 0.


/slurm/home/yrd/liaolab/caohaoxue/anaconda3/envs/my_env/lib/python3.13/site-packages/scanpy/preprocessing/_pca/__init__.py:438: FutureWarning: Argument `use_highly_variable` is deprecated, consider using the mask argument. Use_highly_variable=True can be called through mask_var="highly_variable". Use_highly_variable=False can be called through mask_var=None
  warn(msg, FutureWarning)
/slurm/home/yrd/liaolab/caohaoxue/anaconda3/envs/my_env/lib/python3.13/site-packages/scanpy/preprocessing/_pca/__init__.py:438: FutureWarning: Argument `use_highly_variable` is deprecated, consider using the mask argument. Use_highly_variable=True can be called through mask_var="highly_variable". Use_highly_variable=False can be called through mask_var=None
  warn(msg, FutureWarning)
/slurm/home/yrd/liaolab/caohaoxue/anaconda3/envs/my_env/lib/python3.13/site-packages/scanpy/preprocessing/_pca/__init__.py:438: FutureWarning: Argument `use_highly_variable` is deprecated, consider using the mask argument. Use

In [53]:
kbet = pd.DataFrame(data = [kbet_score_cyto,kbet_score_spla,kbet_score_simp,kbet_score_scde,kbet_score_scva],
                   index = ['CytOrigin', 'Splatter', 'Splatter-Simple', 'scDesign3', 'scVAEDer'],
                   columns = ['E12'])
kbet.to_csv('/slurm/home/yrd/liaolab/caohaoxue/embryo_work/benchmark/E12_kbet.csv')

In [121]:
asw_score_cyto = silhouette_batch(
    adata[adata.obs.time.isin(['E12','E12-CytOrigin'])],
    batch_key="source",
    group_key="stage",
    embed="X_umap",
    metric="euclidean"
)

asw_score_scde = silhouette_batch(
    adata[adata.obs.time.isin(['E12','E12-scDesign3'])],
    batch_key="source",
    group_key="stage",
    embed="X_umap",
    metric="euclidean"
)

asw_score_scva = silhouette_batch(
    adata[adata.obs.time.isin(['E12','E12-scVAEDer'])],
    batch_key="source",
    group_key="stage",
    embed="X_umap",
    metric="euclidean"
)

asw_score_spla = silhouette_batch(
    adata[adata.obs.time.isin(['E12','E12-Splatter'])],
    batch_key="source",
    group_key="stage",
    embed="X_umap",
    metric="euclidean"
)

asw_score_simp = silhouette_batch(
    adata[adata.obs.time.isin(['E12','E12-Splatter-Simple'])],
    batch_key="source",
    group_key="stage",
    embed="X_umap",
    metric="euclidean"
)

mean silhouette per group:        silhouette_score
group                  
E12             0.80211
mean silhouette per group:        silhouette_score
group                  
E12            0.785801
mean silhouette per group:        silhouette_score
group                  
E12            0.631209
mean silhouette per group:        silhouette_score
group                  
E12            0.363142
mean silhouette per group:        silhouette_score
group                  
E12            0.200342


/slurm/home/yrd/liaolab/caohaoxue/anaconda3/envs/my_env/lib/python3.13/site-packages/IPython/core/interactiveshell.py:3667: FutureWarning: In function `silhouette_batch`, argument `group_key` was renamed to `label_key`.
  exec(code_obj, self.user_global_ns, self.user_ns)
/slurm/home/yrd/liaolab/caohaoxue/anaconda3/envs/my_env/lib/python3.13/site-packages/IPython/core/interactiveshell.py:3667: FutureWarning: In function `silhouette_batch`, argument `group_key` was renamed to `label_key`.
  exec(code_obj, self.user_global_ns, self.user_ns)
/slurm/home/yrd/liaolab/caohaoxue/anaconda3/envs/my_env/lib/python3.13/site-packages/IPython/core/interactiveshell.py:3667: FutureWarning: In function `silhouette_batch`, argument `group_key` was renamed to `label_key`.
  exec(code_obj, self.user_global_ns, self.user_ns)
/slurm/home/yrd/liaolab/caohaoxue/anaconda3/envs/my_env/lib/python3.13/site-packages/IPython/core/interactiveshell.py:3667: FutureWarning: In function `silhouette_batch`, argument `gro

In [127]:
asw = pd.DataFrame(data = [asw_score_cyto,asw_score_spla,asw_score_simp,asw_score_scde,asw_score_scva],
                   index = ['CytOrigin', 'Splatter', 'Splatter-Simple', 'scDesign3', 'scVAEDer'],
                   columns = ['E12'])
asw.to_csv('/slurm/home/yrd/liaolab/caohaoxue/embryo_work/benchmark/E12_asw.csv')

# E14

In [55]:
cyto = sc.read_h5ad('/slurm/home/yrd/liaolab/caohaoxue/embryo_work/benchmark/simulated_matrix/h5ad_format/E14f_CytO.h5ad')
spla = sc.read_h5ad('/slurm/home/yrd/liaolab/caohaoxue/embryo_work/benchmark/simulated_matrix/h5ad_format/E14f_splatter.h5ad')
simp = sc.read_h5ad('/slurm/home/yrd/liaolab/caohaoxue/embryo_work/benchmark/simulated_matrix/h5ad_format/E14f_splatter_simple.h5ad')
scde = sc.read_h5ad('/slurm/home/yrd/liaolab/caohaoxue/embryo_work/benchmark/simulated_matrix/h5ad_format/E14f_scDesign3.h5ad')
scva = sc.read_h5ad('/slurm/home/yrd/liaolab/caohaoxue/embryo_work/benchmark/simulated_matrix/h5ad_format/E14f_scVAEDer.h5ad')
real = sc.read_h5ad('/slurm/home/yrd/liaolab/caohaoxue/embryo_work/data_standard/Zygote_to_CS7.h5ad')

/slurm/home/yrd/liaolab/caohaoxue/anaconda3/envs/my_env/lib/python3.13/site-packages/anndata/_core/anndata.py:1756: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [56]:
real = real[real.obs.time == 'E14']
real.var = real.var.rename(columns = {'x':'features'})
real.var.index = real.var.features

In [61]:
adata_int = sc.concat([cyto,scde,spla,simp,scva],
                      join = 'outer',
                      fill_value = 0)

adata = sc.concat([real,adata_int],
                 label = 'source',
                 keys = ['real','simulated'],
                 index_unique = '-')

adata_raw = sc.concat([real,adata_int],
                 label = 'source',
                 keys = ['real','simulated'],
                 index_unique = '-')

/slurm/home/yrd/liaolab/caohaoxue/anaconda3/envs/my_env/lib/python3.13/site-packages/anndata/_core/anndata.py:1756: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/slurm/home/yrd/liaolab/caohaoxue/anaconda3/envs/my_env/lib/python3.13/site-packages/anndata/_core/anndata.py:1756: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/slurm/home/yrd/liaolab/caohaoxue/anaconda3/envs/my_env/lib/python3.13/site-packages/anndata/_core/anndata.py:1756: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [62]:
sc.tl.pca(adata, svd_solver='arpack')
adata.obs['stage'] = 'E14'
sc.external.pp.bbknn(adata, batch_key='source')
sc.tl.umap(adata)

sc.tl.pca(adata_raw, svd_solver='arpack')
adata_raw.obs['stage'] = 'E14'
sc.pp.neighbors(adata_raw)
sc.tl.umap(adata_raw)

/slurm/home/yrd/liaolab/caohaoxue/anaconda3/envs/my_env/lib/python3.13/site-packages/sklearn/manifold/_spectral_embedding.py:329: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(


In [45]:
asw_score_cyto = silhouette_batch(
    adata[adata.obs.time.isin(['E14','E14-CytOrigin'])],
    batch_key="source",
    group_key="stage",
    embed="X_umap",
    metric="euclidean"
)

asw_score_scde = silhouette_batch(
    adata[adata.obs.time.isin(['E14','E14-scDesign3'])],
    batch_key="source",
    group_key="stage",
    embed="X_umap",
    metric="euclidean"
)

asw_score_scva = silhouette_batch(
    adata[adata.obs.time.isin(['E14','E14-scVAEDer'])],
    batch_key="source",
    group_key="stage",
    embed="X_umap",
    metric="euclidean"
)

asw_score_spla = silhouette_batch(
    adata[adata.obs.time.isin(['E14','E14-Splatter'])],
    batch_key="source",
    group_key="stage",
    embed="X_umap",
    metric="euclidean"
)

asw_score_simp = silhouette_batch(
    adata[adata.obs.time.isin(['E14','E14-Splatter-Simple'])],
    batch_key="source",
    group_key="stage",
    embed="X_umap",
    metric="euclidean"
)


mean silhouette per group:        silhouette_score
group                  
E14            0.826259
mean silhouette per group:        silhouette_score
group                  
E14            0.733383
mean silhouette per group:        silhouette_score
group                  
E14            0.577555
mean silhouette per group:        silhouette_score
group                  
E14             0.29796
mean silhouette per group:        silhouette_score
group                  
E14            0.184581


/slurm/home/yrd/liaolab/caohaoxue/anaconda3/envs/my_env/lib/python3.13/site-packages/IPython/core/interactiveshell.py:3667: FutureWarning: In function `silhouette_batch`, argument `group_key` was renamed to `label_key`.
  exec(code_obj, self.user_global_ns, self.user_ns)
/slurm/home/yrd/liaolab/caohaoxue/anaconda3/envs/my_env/lib/python3.13/site-packages/IPython/core/interactiveshell.py:3667: FutureWarning: In function `silhouette_batch`, argument `group_key` was renamed to `label_key`.
  exec(code_obj, self.user_global_ns, self.user_ns)
/slurm/home/yrd/liaolab/caohaoxue/anaconda3/envs/my_env/lib/python3.13/site-packages/IPython/core/interactiveshell.py:3667: FutureWarning: In function `silhouette_batch`, argument `group_key` was renamed to `label_key`.
  exec(code_obj, self.user_global_ns, self.user_ns)
/slurm/home/yrd/liaolab/caohaoxue/anaconda3/envs/my_env/lib/python3.13/site-packages/IPython/core/interactiveshell.py:3667: FutureWarning: In function `silhouette_batch`, argument `gro

In [46]:
asw = pd.DataFrame(data = [asw_score_cyto,asw_score_spla,asw_score_simp,asw_score_scde,asw_score_scva],
                   index = ['CytOrigin', 'Splatter', 'Splatter-Simple', 'scDesign3', 'scVAEDer'],
                   columns = ['E14'])
asw.to_csv('/slurm/home/yrd/liaolab/caohaoxue/embryo_work/benchmark/E14_asw.csv')

In [63]:
kbet_score_cyto = pcr_comparison(
    adata_raw[adata_raw.obs.time.isin(['E14','E14-CytOrigin'])],
    adata[adata.obs.time.isin(['E14','E14-CytOrigin'])],
    embed="X_umap",
    covariate='time'
)

kbet_score_scde = pcr_comparison(
    adata_raw[adata_raw.obs.time.isin(['E14','E14-scDesign3'])],
    adata[adata.obs.time.isin(['E14','E14-scDesign3'])],
    embed="X_umap",
    covariate='time'
)

kbet_score_scva = pcr_comparison(
    adata_raw[adata_raw.obs.time.isin(['E14','E14-scVAEDer'])],
    adata[adata.obs.time.isin(['E14','E14-scVAEDer'])],
    embed="X_umap",
    covariate='time'
)

kbet_score_spla = pcr_comparison(
    adata_raw[adata_raw.obs.time.isin(['E14','E14-Splatter'])],
    adata[adata.obs.time.isin(['E14','E14-Splatter'])],
    embed="X_umap",
    covariate='time'
)

kbet_score_simp = pcr_comparison(
    adata_raw[adata_raw.obs.time.isin(['E14','E14-Splatter-Simple'])],
    adata[adata.obs.time.isin(['E14','E14-Splatter-Simple'])],
    embed="X_umap",
    covariate='time'
)

/slurm/home/yrd/liaolab/caohaoxue/anaconda3/envs/my_env/lib/python3.13/site-packages/scanpy/preprocessing/_pca/__init__.py:438: FutureWarning: Argument `use_highly_variable` is deprecated, consider using the mask argument. Use_highly_variable=True can be called through mask_var="highly_variable". Use_highly_variable=False can be called through mask_var=None
  warn(msg, FutureWarning)
/slurm/home/yrd/liaolab/caohaoxue/anaconda3/envs/my_env/lib/python3.13/site-packages/scanpy/preprocessing/_pca/__init__.py:438: FutureWarning: Argument `use_highly_variable` is deprecated, consider using the mask argument. Use_highly_variable=True can be called through mask_var="highly_variable". Use_highly_variable=False can be called through mask_var=None
  warn(msg, FutureWarning)
/slurm/home/yrd/liaolab/caohaoxue/anaconda3/envs/my_env/lib/python3.13/site-packages/scanpy/preprocessing/_pca/__init__.py:438: FutureWarning: Argument `use_highly_variable` is deprecated, consider using the mask argument. Use

In [64]:
kbet = pd.DataFrame(data = [kbet_score_cyto,kbet_score_spla,kbet_score_simp,kbet_score_scde,kbet_score_scva],
                   index = ['CytOrigin', 'Splatter', 'Splatter-Simple', 'scDesign3', 'scVAEDer'],
                   columns = ['E14'])
kbet.to_csv('/slurm/home/yrd/liaolab/caohaoxue/embryo_work/benchmark/E14_kbet.csv')

# CS10

In [12]:
cyto = sc.read_h5ad('/slurm/home/yrd/liaolab/caohaoxue/embryo_work/benchmark/simulated_matrix/h5ad_format/CS10f_CytO.h5ad')
spla = sc.read_h5ad('/slurm/home/yrd/liaolab/caohaoxue/embryo_work/benchmark/simulated_matrix/h5ad_format/CS10f_splatter.h5ad')
simp = sc.read_h5ad('/slurm/home/yrd/liaolab/caohaoxue/embryo_work/benchmark/simulated_matrix/h5ad_format/CS10f_splatter_simple.h5ad')
scde = sc.read_h5ad('/slurm/home/yrd/liaolab/caohaoxue/embryo_work/benchmark/simulated_matrix/h5ad_format/CS10f_scDesign3.h5ad')
scva = sc.read_h5ad('/slurm/home/yrd/liaolab/caohaoxue/embryo_work/benchmark/simulated_matrix/h5ad_format/CS10f_scVAEDer.h5ad')
real = sc.read_h5ad('/slurm/home/yrd/liaolab/caohaoxue/embryo_work/data_standard/CS10.h5ad')

/slurm/home/yrd/liaolab/caohaoxue/anaconda3/envs/my_env/lib/python3.13/site-packages/anndata/_core/anndata.py:1756: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [13]:
real.var = real.var.rename(columns = {'x':'features'})
real.var.index = real.var.features
real.obs['time'] = 'CS10'

In [14]:
adata_int = sc.concat([cyto,scva,spla,simp,scde], join = 'outer', fill_value = 0)

adata = sc.concat([real,adata_int], label = 'source', keys = ['real','simulated'], index_unique = '-')
adata_raw = sc.concat([real,adata_int],label = 'source',keys = ['real','simulated'],index_unique = '-')

sc.tl.pca(adata, svd_solver='arpack')
adata.obs['stage'] = 'CS10'
sc.external.pp.bbknn(adata, batch_key='source')
sc.tl.umap(adata)
sc.tl.embedding_density(adata,groupby='time')

sc.tl.pca(adata_raw, svd_solver='arpack')
adata_raw.obs['stage'] = 'CS10'
sc.pp.neighbors(adata_raw)
sc.tl.umap(adata_raw)

/slurm/home/yrd/liaolab/caohaoxue/anaconda3/envs/my_env/lib/python3.13/site-packages/anndata/_core/anndata.py:1756: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/slurm/home/yrd/liaolab/caohaoxue/anaconda3/envs/my_env/lib/python3.13/site-packages/anndata/_core/anndata.py:1756: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/slurm/home/yrd/liaolab/caohaoxue/anaconda3/envs/my_env/lib/python3.13/site-packages/anndata/_core/anndata.py:1756: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


/slurm/home/yrd/liaolab/caohaoxue/anaconda3/envs/my_env/lib/python3.13/site-packages/sklearn/manifold/_spectral_embedding.py:329: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(
/slurm/home/yrd/liaolab/caohaoxue/anaconda3/envs/my_env/lib/python3.13/site-packages/sklearn/manifold/_spectral_embedding.py:329: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(


In [18]:
asw_score_cyto = silhouette_batch(
    adata[adata.obs.time.isin(['CS10','CS10-CytOrigin'])],
    batch_key="source",
    group_key="stage",
    embed="X_umap",
    metric="euclidean"
)

asw_score_scde = silhouette_batch(
    adata[adata.obs.time.isin(['CS10','CS10-scDesign3'])],
    batch_key="source",
    group_key="stage",
    embed="X_umap",
    metric="euclidean"
)

asw_score_scva = silhouette_batch(
    adata[adata.obs.time.isin(['CS10','CS10-scVAEDer'])],
    batch_key="source",
    group_key="stage",
    embed="X_umap",
    metric="euclidean"
)

asw_score_spla = silhouette_batch(
    adata[adata.obs.time.isin(['CS10','CS10-Splatter'])],
    batch_key="source",
    group_key="stage",
    embed="X_umap",
    metric="euclidean"
)

asw_score_simp = silhouette_batch(
    adata[adata.obs.time.isin(['CS10','CS10-Splatter-Simple'])],
    batch_key="source",
    group_key="stage",
    embed="X_umap",
    metric="euclidean"
)

/slurm/home/yrd/liaolab/caohaoxue/anaconda3/envs/my_env/lib/python3.13/site-packages/IPython/core/interactiveshell.py:3667: FutureWarning: In function `silhouette_batch`, argument `group_key` was renamed to `label_key`.
  exec(code_obj, self.user_global_ns, self.user_ns)


mean silhouette per group:        silhouette_score
group                  
CS10           0.638413


/slurm/home/yrd/liaolab/caohaoxue/anaconda3/envs/my_env/lib/python3.13/site-packages/IPython/core/interactiveshell.py:3667: FutureWarning: In function `silhouette_batch`, argument `group_key` was renamed to `label_key`.
  exec(code_obj, self.user_global_ns, self.user_ns)


mean silhouette per group:        silhouette_score
group                  
CS10           0.481616


/slurm/home/yrd/liaolab/caohaoxue/anaconda3/envs/my_env/lib/python3.13/site-packages/IPython/core/interactiveshell.py:3667: FutureWarning: In function `silhouette_batch`, argument `group_key` was renamed to `label_key`.
  exec(code_obj, self.user_global_ns, self.user_ns)


mean silhouette per group:        silhouette_score
group                  
CS10            0.37638


/slurm/home/yrd/liaolab/caohaoxue/anaconda3/envs/my_env/lib/python3.13/site-packages/IPython/core/interactiveshell.py:3667: FutureWarning: In function `silhouette_batch`, argument `group_key` was renamed to `label_key`.
  exec(code_obj, self.user_global_ns, self.user_ns)


mean silhouette per group:        silhouette_score
group                  
CS10           0.566478


/slurm/home/yrd/liaolab/caohaoxue/anaconda3/envs/my_env/lib/python3.13/site-packages/IPython/core/interactiveshell.py:3667: FutureWarning: In function `silhouette_batch`, argument `group_key` was renamed to `label_key`.
  exec(code_obj, self.user_global_ns, self.user_ns)


mean silhouette per group:        silhouette_score
group                  
CS10           0.309528


In [26]:
asw = pd.DataFrame(data = [asw_score_cyto,asw_score_spla,asw_score_simp,asw_score_scde,asw_score_scva],
                   index = ['CytOrigin', 'Splatter', 'Splatter-Simple', 'scDesign3', 'scVAEDer'],
                   columns = ['CS10'])
asw.to_csv('/slurm/home/yrd/liaolab/caohaoxue/embryo_work/benchmark/CS10_asw.csv')

In [15]:
kbet_score_cyto = pcr_comparison(
    adata_raw[adata_raw.obs.time.isin(['CS10','CS10-CytOrigin'])],
    adata[adata.obs.time.isin(['CS10','CS10-CytOrigin'])],
    embed="X_umap",
    covariate='time'
)

kbet_score_scde = pcr_comparison(
    adata_raw[adata_raw.obs.time.isin(['CS10','CS10-scDesign3'])],
    adata[adata.obs.time.isin(['CS10','CS10-scDesign3'])],
    embed="X_umap",
    covariate='time'
)

kbet_score_scva = pcr_comparison(
    adata_raw[adata_raw.obs.time.isin(['CS10','CS10-scVAEDer'])],
    adata[adata.obs.time.isin(['CS10','CS10-scVAEDer'])],
    embed="X_umap",
    covariate='time'
)

kbet_score_spla = pcr_comparison(
    adata_raw[adata_raw.obs.time.isin(['CS10','CS10-Splatter'])],
    adata[adata.obs.time.isin(['CS10','CS10-Splatter'])],
    embed="X_umap",
    covariate='time'
)

kbet_score_simp = pcr_comparison(
    adata_raw[adata_raw.obs.time.isin(['CS10','CS10-Splatter-Simple'])],
    adata[adata.obs.time.isin(['CS10','CS10-Splatter-Simple'])],
    embed="X_umap",
    covariate='time'
)

/slurm/home/yrd/liaolab/caohaoxue/anaconda3/envs/my_env/lib/python3.13/site-packages/scanpy/preprocessing/_pca/__init__.py:438: FutureWarning: Argument `use_highly_variable` is deprecated, consider using the mask argument. Use_highly_variable=True can be called through mask_var="highly_variable". Use_highly_variable=False can be called through mask_var=None
  warn(msg, FutureWarning)
/slurm/home/yrd/liaolab/caohaoxue/anaconda3/envs/my_env/lib/python3.13/site-packages/scanpy/preprocessing/_pca/__init__.py:438: FutureWarning: Argument `use_highly_variable` is deprecated, consider using the mask argument. Use_highly_variable=True can be called through mask_var="highly_variable". Use_highly_variable=False can be called through mask_var=None
  warn(msg, FutureWarning)
/slurm/home/yrd/liaolab/caohaoxue/anaconda3/envs/my_env/lib/python3.13/site-packages/scanpy/preprocessing/_pca/__init__.py:438: FutureWarning: Argument `use_highly_variable` is deprecated, consider using the mask argument. Use

Variance contribution increased after integration!
Setting PCR comparison score to 0.


/slurm/home/yrd/liaolab/caohaoxue/anaconda3/envs/my_env/lib/python3.13/site-packages/scanpy/preprocessing/_pca/__init__.py:438: FutureWarning: Argument `use_highly_variable` is deprecated, consider using the mask argument. Use_highly_variable=True can be called through mask_var="highly_variable". Use_highly_variable=False can be called through mask_var=None
  warn(msg, FutureWarning)


In [16]:
kbet = pd.DataFrame(data = [kbet_score_cyto,kbet_score_spla,kbet_score_simp,kbet_score_scde,kbet_score_scva],
                   index = ['CytOrigin', 'Splatter', 'Splatter-Simple', 'scDesign3', 'scVAEDer'],
                   columns = ['CS10'])
kbet.to_csv('/slurm/home/yrd/liaolab/caohaoxue/embryo_work/benchmark/CS10_kbet.csv')

# CS11

In [4]:
cyto = sc.read_h5ad('/slurm/home/yrd/liaolab/caohaoxue/embryo_work/benchmark/simulated_matrix/h5ad_format/CS11f_CytO.h5ad')
spla = sc.read_h5ad('/slurm/home/yrd/liaolab/caohaoxue/embryo_work/benchmark/simulated_matrix/h5ad_format/CS11f_splatter.h5ad')
simp = sc.read_h5ad('/slurm/home/yrd/liaolab/caohaoxue/embryo_work/benchmark/simulated_matrix/h5ad_format/CS11f_splatter_simple.h5ad')
scde = sc.read_h5ad('/slurm/home/yrd/liaolab/caohaoxue/embryo_work/benchmark/simulated_matrix/h5ad_format/CS11f_scDesign3.h5ad')
scva = sc.read_h5ad('/slurm/home/yrd/liaolab/caohaoxue/embryo_work/benchmark/simulated_matrix/h5ad_format/CS11f_scVAEDer.h5ad')
real = sc.read_h5ad('/slurm/home/yrd/liaolab/caohaoxue/embryo_work/data_standard/CS11.h5ad')

In [6]:
adata_int = sc.concat([cyto,scde,scva,spla,simp],
                      join = 'outer',
                      fill_value = 0)

adata = sc.concat([real,adata_int],
                 label = 'source',
                 keys = ['real','simulated'],
                 index_unique = '-')

adata_raw = sc.concat([real,adata_int],
                 label = 'source',
                 keys = ['real','simulated'],
                 index_unique = '-')

sc.tl.pca(adata, svd_solver='arpack')
adata.obs['stage'] = 'CS11'
sc.external.pp.bbknn(adata, batch_key='source')
sc.tl.umap(adata)
sc.tl.embedding_density(adata,groupby='time')

sc.tl.pca(adata_raw, svd_solver='arpack')
adata_raw.obs['stage'] = 'CS11'
sc.pp.neighbors(adata_raw)
sc.tl.umap(adata_raw)

/slurm/home/yrd/liaolab/caohaoxue/anaconda3/envs/my_env/lib/python3.13/site-packages/anndata/_core/anndata.py:1756: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/slurm/home/yrd/liaolab/caohaoxue/anaconda3/envs/my_env/lib/python3.13/site-packages/anndata/_core/merge.py:1349: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  concat_annot = pd.concat(
/slurm/home/yrd/liaolab/caohaoxue/anaconda3/envs/my_env/lib/python3.13/site-packages/anndata/_core/anndata.py:1756: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/slurm/home/yrd/liaolab/caohaoxue/anaconda3/envs/my_env/lib/

/slurm/home/yrd/liaolab/caohaoxue/anaconda3/envs/my_env/lib/python3.13/site-packages/sklearn/manifold/_spectral_embedding.py:329: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(
/slurm/home/yrd/liaolab/caohaoxue/anaconda3/envs/my_env/lib/python3.13/site-packages/sklearn/manifold/_spectral_embedding.py:329: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(


In [43]:
asw_score_cyto = silhouette_batch(
    adata[adata.obs.time.isin(['CS11','CS11-CytOrigin'])],
    batch_key="source",
    group_key="stage",
    embed="X_umap",
    metric="euclidean"
)

asw_score_scde = silhouette_batch(
    adata[adata.obs.time.isin(['CS11','CS11-scDesign3'])],
    batch_key="source",
    group_key="stage",
    embed="X_umap",
    metric="euclidean"
)

asw_score_scva = silhouette_batch(
    adata[adata.obs.time.isin(['CS11','CS11-scVAEDer'])],
    batch_key="source",
    group_key="stage",
    embed="X_umap",
    metric="euclidean"
)

asw_score_spla = silhouette_batch(
    adata[adata.obs.time.isin(['CS11','CS11-Splatter'])],
    batch_key="source",
    group_key="stage",
    embed="X_umap",
    metric="euclidean"
)

asw_score_simp = silhouette_batch(
    adata[adata.obs.time.isin(['CS11','CS11-Splatter-Simple'])],
    batch_key="source",
    group_key="stage",
    embed="X_umap",
    metric="euclidean"
)


/slurm/home/yrd/liaolab/caohaoxue/anaconda3/envs/my_env/lib/python3.13/site-packages/IPython/core/interactiveshell.py:3667: FutureWarning: In function `silhouette_batch`, argument `group_key` was renamed to `label_key`.
  exec(code_obj, self.user_global_ns, self.user_ns)


mean silhouette per group:        silhouette_score
group                  
CS11           0.792631


/slurm/home/yrd/liaolab/caohaoxue/anaconda3/envs/my_env/lib/python3.13/site-packages/IPython/core/interactiveshell.py:3667: FutureWarning: In function `silhouette_batch`, argument `group_key` was renamed to `label_key`.
  exec(code_obj, self.user_global_ns, self.user_ns)


mean silhouette per group:        silhouette_score
group                  
CS11           0.613635


/slurm/home/yrd/liaolab/caohaoxue/anaconda3/envs/my_env/lib/python3.13/site-packages/IPython/core/interactiveshell.py:3667: FutureWarning: In function `silhouette_batch`, argument `group_key` was renamed to `label_key`.
  exec(code_obj, self.user_global_ns, self.user_ns)


mean silhouette per group:        silhouette_score
group                  
CS11           0.808797


/slurm/home/yrd/liaolab/caohaoxue/anaconda3/envs/my_env/lib/python3.13/site-packages/IPython/core/interactiveshell.py:3667: FutureWarning: In function `silhouette_batch`, argument `group_key` was renamed to `label_key`.
  exec(code_obj, self.user_global_ns, self.user_ns)


mean silhouette per group:        silhouette_score
group                  
CS11           0.433011


/slurm/home/yrd/liaolab/caohaoxue/anaconda3/envs/my_env/lib/python3.13/site-packages/IPython/core/interactiveshell.py:3667: FutureWarning: In function `silhouette_batch`, argument `group_key` was renamed to `label_key`.
  exec(code_obj, self.user_global_ns, self.user_ns)


mean silhouette per group:        silhouette_score
group                  
CS11           0.427267


In [46]:
asw = pd.DataFrame(data = [asw_score_cyto,asw_score_spla,asw_score_simp,asw_score_scde,asw_score_scva],
                   index = ['CytOrigin', 'Splatter', 'Splatter-Simple', 'scDesign3', 'scVAEDer'],
                   columns = ['CS11'])
asw.to_csv('/slurm/home/yrd/liaolab/caohaoxue/embryo_work/benchmark/CS11_asw.csv')

In [7]:
kbet_score_cyto = pcr_comparison(
    adata_raw[adata_raw.obs.time.isin(['CS11','CS11-CytOrigin'])],
    adata[adata.obs.time.isin(['CS11','CS11-CytOrigin'])],
    embed="X_umap",
    covariate='time'
)

kbet_score_scde = pcr_comparison(
    adata_raw[adata_raw.obs.time.isin(['CS11','CS11-scDesign3'])],
    adata[adata.obs.time.isin(['CS11','CS11-scDesign3'])],
    embed="X_umap",
    covariate='time'
)

kbet_score_scva = pcr_comparison(
    adata_raw[adata_raw.obs.time.isin(['CS11','CS11-scVAEDer'])],
    adata[adata.obs.time.isin(['CS11','CS11-scVAEDer'])],
    embed="X_umap",
    covariate='time'
)

kbet_score_spla = pcr_comparison(
    adata_raw[adata_raw.obs.time.isin(['CS11','CS11-Splatter'])],
    adata[adata.obs.time.isin(['CS11','CS11-Splatter'])],
    embed="X_umap",
    covariate='time'
)

kbet_score_simp = pcr_comparison(
    adata_raw[adata_raw.obs.time.isin(['CS11','CS11-Splatter-Simple'])],
    adata[adata.obs.time.isin(['CS11','CS11-Splatter-Simple'])],
    embed="X_umap",
    covariate='time'
)

/slurm/home/yrd/liaolab/caohaoxue/anaconda3/envs/my_env/lib/python3.13/site-packages/scanpy/preprocessing/_pca/__init__.py:438: FutureWarning: Argument `use_highly_variable` is deprecated, consider using the mask argument. Use_highly_variable=True can be called through mask_var="highly_variable". Use_highly_variable=False can be called through mask_var=None
  warn(msg, FutureWarning)
/slurm/home/yrd/liaolab/caohaoxue/anaconda3/envs/my_env/lib/python3.13/site-packages/scanpy/preprocessing/_pca/__init__.py:438: FutureWarning: Argument `use_highly_variable` is deprecated, consider using the mask argument. Use_highly_variable=True can be called through mask_var="highly_variable". Use_highly_variable=False can be called through mask_var=None
  warn(msg, FutureWarning)
/slurm/home/yrd/liaolab/caohaoxue/anaconda3/envs/my_env/lib/python3.13/site-packages/scanpy/preprocessing/_pca/__init__.py:438: FutureWarning: Argument `use_highly_variable` is deprecated, consider using the mask argument. Use

In [11]:
kbet = pd.DataFrame(data = [kbet_score_cyto,kbet_score_spla,kbet_score_simp,kbet_score_scde,kbet_score_scva],
                   index = ['CytOrigin', 'Splatter', 'Splatter-Simple', 'scDesign3', 'scVAEDer'],
                   columns = ['CS11'])
kbet.to_csv('/slurm/home/yrd/liaolab/caohaoxue/embryo_work/benchmark/CS11_kbet.csv')